In [1]:
!pip install keras==2.2.4

Looking in indexes: http://mirrors.tencentyun.com/pypi/simple
     |████████████████████████████████| 312 kB 897 kB/s eta 0:00:01


In [2]:
from keras.models import Model
from keras.optimizers import SGD,Adam,RMSprop
# from keras.layers import Dense, Input, LSTM, Embedding,Dropout,Bidirectional,Flatten
from keras.layers import *
import os

# from __future__ import print_function
from keras import backend as K
from keras.engine.topology import Layer
import h5py

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
import pandas as pd
import numpy as np
import keras

Using TensorFlow backend.


In [3]:
# Position_Embedding
#! -*- coding: utf-8 -*-
#%%

class Position_Embedding(Layer):
 
    def __init__(self, size=None, mode='sum', **kwargs):
        self.size = size #必须为偶数
        self.mode = mode
        super(Position_Embedding, self).__init__(**kwargs)
 
    def call(self, x):
        if (self.size == None) or (self.mode == 'sum'):
            self.size = int(x.shape[-1])
        batch_size,seq_len = K.shape(x)[0],K.shape(x)[1]
        position_j = 1. / K.pow(10000., \
                                 2 * K.arange(self.size / 2, dtype='float32' \
                               ) / self.size)
        position_j = K.expand_dims(position_j, 0)
        position_i = K.cumsum(K.ones_like(x[:,:,0]), 1)-1 #K.arange不支持变长，只好用这种方法生成
        position_i = K.expand_dims(position_i, 2)
        position_ij = K.dot(position_i, position_j)
        position_ij = K.concatenate([K.cos(position_ij), K.sin(position_ij)], 2)
        if self.mode == 'sum':
            return position_ij + x
        elif self.mode == 'concat':
            return K.concatenate([position_ij, x], 2)
 
    def compute_output_shape(self, input_shape):
        if self.mode == 'sum':
            return input_shape
        elif self.mode == 'concat':
            return (input_shape[0], input_shape[1], input_shape[2]+self.size)

In [4]:
# attention

class Attention(Layer):
 
    def __init__(self, nb_head, size_per_head, **kwargs):
        self.nb_head = nb_head
        self.size_per_head = size_per_head
        self.output_dim = nb_head*size_per_head
        super(Attention, self).__init__(**kwargs)
 
    def build(self, input_shape):
        self.WQ = self.add_weight(name='WQ',
                                  shape=(input_shape[0][-1], self.output_dim),
                                  initializer='glorot_uniform',
                                  trainable=True)
        self.WK = self.add_weight(name='WK',
                                  shape=(input_shape[1][-1], self.output_dim),
                                  initializer='glorot_uniform',
                                  trainable=True)
        self.WV = self.add_weight(name='WV',
                                  shape=(input_shape[2][-1], self.output_dim),
                                  initializer='glorot_uniform',
                                  trainable=True)
        super(Attention, self).build(input_shape)
 
    def Mask(self, inputs, seq_len, mode='mul'):
        if seq_len == None:
            return inputs
        else:
            mask = K.one_hot(seq_len[:,0], K.shape(inputs)[1])
            mask = 1 - K.cumsum(mask, 1)
            for _ in range(len(inputs.shape)-2):
                mask = K.expand_dims(mask, 2)
            if mode == 'mul':
                return inputs * mask
            if mode == 'add':
                return inputs - (1 - mask) * 1e12
 
    def call(self, x):
        #如果只传入Q_seq,K_seq,V_seq，那么就不做Mask
        #如果同时传入Q_seq,K_seq,V_seq,Q_len,V_len，那么对多余部分做Mask
        if len(x) == 3:
            Q_seq,K_seq,V_seq = x
            Q_len,V_len = None,None
        elif len(x) == 5:
            Q_seq,K_seq,V_seq,Q_len,V_len = x
        #对Q、K、V做线性变换
        Q_seq = K.dot(Q_seq, self.WQ)
        Q_seq = K.reshape(Q_seq, (-1, K.shape(Q_seq)[1], self.nb_head, self.size_per_head))
        Q_seq = K.permute_dimensions(Q_seq, (0,2,1,3))
        K_seq = K.dot(K_seq, self.WK)
        K_seq = K.reshape(K_seq, (-1, K.shape(K_seq)[1], self.nb_head, self.size_per_head))
        K_seq = K.permute_dimensions(K_seq, (0,2,1,3))
        V_seq = K.dot(V_seq, self.WV)
        V_seq = K.reshape(V_seq, (-1, K.shape(V_seq)[1], self.nb_head, self.size_per_head))
        V_seq = K.permute_dimensions(V_seq, (0,2,1,3))
        #计算内积，然后mask，然后softmax
        A = K.batch_dot(Q_seq, K_seq, axes=[3,3]) / self.size_per_head**0.5
        A = K.permute_dimensions(A, (0,3,2,1))
        A = self.Mask(A, V_len, 'add')
        A = K.permute_dimensions(A, (0,3,2,1))
        A = K.softmax(A)
        #输出并mask
        O_seq = K.batch_dot(A, V_seq, axes=[3,2])
        O_seq = K.permute_dimensions(O_seq, (0,2,1,3))
        O_seq = K.reshape(O_seq, (-1, K.shape(O_seq)[1], self.output_dim))
        O_seq = self.Mask(O_seq, Q_len, 'mul')
        return O_seq
 
    def compute_output_shape(self, input_shape):
        return (input_shape[0][0], input_shape[0][1], self.output_dim)

In [5]:

def buid_model():
    # LSTM 模型
    print('lstm model start...\n')

    # 标题输入：接收一个含有 200 个整数的序列，每个整数在 1 到 3812202 之间。
    S_inputs1 = Input(shape=(200,),name='main_input1',dtype='int32')
    with h5py.File('../../get_w2v_feat/w2v1_pre_ad/embeddings_matrix_ad_size_32_w200_count_0.h5','r') as f:
        embeddings_matrix1 = np.array(f.get('embeddings_matrix'))
    embeddings1 = Embedding(input_dim = len(embeddings_matrix1), # 字典长度
                                output_dim = 32, # 词向量 长度（32）
                                weights=[embeddings_matrix1], # 重点：预训练的词向量系数
                                input_length=200, # 每句话的 最大长度（必须padding） 
                                trainable=True, # 是否在 训练的过程中 更新词向量
                                mask_zero = False)(S_inputs1)

    S_inputs2 = Input(shape=(200,),name='main_input2',dtype='int32')
    with h5py.File('../../get_w2v_feat/w2v1_pre_advertiser/embeddings_matrix_advertiser_size_32_w200_count_5.h5','r') as f:
        embeddings_matrix2 = np.array(f.get('embeddings_matrix'))
    embeddings2 = Embedding(input_dim = len(embeddings_matrix2), # 字典长度
                                output_dim = 32, # 词向量 长度（32）
                                weights=[embeddings_matrix2], # 重点：预训练的词向量系数
                                input_length=200, # 每句话的 最大长度（必须padding） 
                                trainable=True, # 是否在 训练的过程中 更新词向量
                                mask_zero = False)(S_inputs2)
    
    S_inputs3 = Input(shape=(200,),name='main_input3',dtype='int32')
    with h5py.File('../../get_w2v_feat/w2v1_pre_creative_id/embeddings_matrix_creative_size_32_w200_count_5.h5','r') as f:
        embeddings_matrix3 = np.array(f.get('embeddings_matrix'))
    embeddings3 = Embedding(input_dim = len(embeddings_matrix3), # 字典长度
                                output_dim = 32, # 词向量 长度（32）
                                weights=[embeddings_matrix3], # 重点：预训练的词向量系数
                                input_length=200, # 每句话的 最大长度（必须padding） 
                                trainable=True, # 是否在 训练的过程中 更新词向量
                                mask_zero = False)(S_inputs3)
    

    
#     emb = keras.layers.concatenate([O_seq1, O_seq2, O_seq3])
    emb = keras.layers.concatenate([embeddings1, embeddings2, embeddings3])

    O_seq = Attention(8,4)([emb,emb,emb])
#     O_seq = Attention(8,8)([O_seq,O_seq,O_seq])

    O_seq = GlobalAveragePooling1D()(O_seq)

#     O_seq = Dropout(0.5)(O_seq)#尽量不要用

#     lstm_out = Bidirectional(LSTM(10,activation='softsign',return_sequences=True))(O_seq)
#     lstm_out = GlobalAveragePooling1D()(lstm_out)

    outputs = Dense(10, activation='softmax', name='main_output')(O_seq)

    # 定义一个具有两个输入输出的模型
    model = keras.models.Model(inputs=[S_inputs1,S_inputs2,S_inputs3],#,auxiliary_input],
                               outputs=[outputs])  # 这里的输入输出顺序与fit时一致就好
#     opt = RMSprop(lr=0.01,  clipnorm=1.0)
    opt = Adam(lr=0.005)
    model.compile(optimizer=opt,
                  sample_weight_mode='None',#"temporal",
                  loss={'main_output': 'categorical_crossentropy'},
                 metrics=['accuracy'])
    print(model.summary())
    return model


def data_load():
    print('loading data ... \n')

    with h5py.File('../../get_w2v_feat/w2v1_pre_ad/word_train_ad_w2v_w200.h5', 'r') as f:
        data = np.array(f.get('word_data'))
        
    label = pd.read_csv('../../train_preliminary/user.csv').sort_values(by=['user_id'])

    train_x, test_x, train_y, test_y = train_test_split(data, label, test_size=0.2, random_state=2020)

    train_y_age = train_y['age'].values - 1
    train_y_age = keras.utils.np_utils.to_categorical(train_y_age, num_classes=10)
    train_y_gender = train_y['gender'].values - 1

    test_y_age = test_y['age'].values - 1
    test_y_age = keras.utils.np_utils.to_categorical(test_y_age, num_classes=10)
    test_y_gender = test_y['gender'].values - 1

    print('get data ... \n')

    return train_x, test_x, train_y_age, train_y_gender,test_y_age,test_y_gender

def load_data2():
    
    with h5py.File('../../get_w2v_feat/w2v1_pre_advertiser/word_train_advertiser_w2v_w200.h5', 'r') as f:
        data = np.array(f.get('word_data'))

    train_x, test_x= train_test_split(data, test_size=0.2, random_state=2020)
    return train_x, test_x

def load_data3():
    
    with h5py.File('../../get_w2v_feat/w2v1_pre_creative_id/word_train_creative_w2v_w200.h5', 'r') as f:
        data = np.array(f.get('word_data'))

    train_x, test_x= train_test_split(data, test_size=0.2, random_state=2020)
    return train_x, test_x



In [6]:
def get_filename_for_saving(save_dir):
    return os.path.join(save_dir,
                        "attention_comb_age_aver_w2v200_{val_loss:.3f}-{val_acc:.3f}-{epoch:03d}-{loss:.3f}-{acc:.3f}.hdf5")


model = buid_model()
print('lstm model geted...\n')

lstm model start...







__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
main_input1 (InputLayer)        (None, 200)          0                                            
__________________________________________________________________________________________________
main_input2 (InputLayer)        (None, 200)          0                                            
__________________________________________________________________________________________________
main_input3 (InputLayer)        (None, 200)          0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 200, 32)      32319296    main_input1[0][0]                
__________________________________________________________________________________

In [14]:
print(model.summary())

Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
main_input (InputLayer)      (None, 200)               0         
_________________________________________________________________
embedding_2 (Embedding)      (None, 200, 15)           944490    
_________________________________________________________________
lstm_2 (LSTM)                (None, 10)                1040      
_________________________________________________________________
main_output (Dense)          (None, 10)                110       
Total params: 945,640
Trainable params: 945,640
Non-trainable params: 0
_________________________________________________________________
None


In [8]:
train_x, test_x, train_y_age, train_y_gender,test_y_age,test_y_gender = data_load()

train_x2, test_x2 = load_data2()

train_x3, test_x3 = load_data3()

loading data ... 

get data ... 



In [ ]:
print('lstm model fit...\n')
checkpointer = keras.callbacks.ModelCheckpoint(
    filepath=get_filename_for_saving(''),
    save_best_only=False)
stopping = keras.callbacks.EarlyStopping(patience=8)
reduce_lr = keras.callbacks.ReduceLROnPlateau(factor=0.1, patience=2, min_lr=0.0001)

model.fit({'main_input1': train_x ,'main_input2': train_x2,'main_input3': train_x3},
          {'main_output': train_y_age},
          epochs=100,
          batch_size=256,
          validation_data=({'main_input1': test_x,'main_input2': test_x2,'main_input3': test_x3},
                           {'main_output': test_y_age}),
          callbacks=[checkpointer, reduce_lr, stopping])

lstm model fit...

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Train on 720000 samples, validate on 180000 samples
Epoch 1/100
720000/720000 [==============================] - 3421s 5ms/step - loss: 1.3766 - acc: 0.4376 - val_loss: 1.3121 - val_acc: 0.4635
Epoch 2/100
 35840/720000 [>.............................] - ETA: 50:06 - loss: 0.9789 - acc: 0.6189

In [ ]:



# pre = model.predict(test_x,verbose=1)

# #评估结果
# from sklearn.metrics import confusion_matrix, classification_report
# y_ = np.reshape(np.argmax(test_y,axis=1),[-1])
# pre_ = np.reshape(np.argmax(pre, axis=1),[-1])
# #每个类的各项指标
# cm = confusion_matrix(y_, pre_)
# # np.set_printoptions(precision=3)
# cm_normalized = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
# print(cm_normalized)
# print(classification_report(y_, pre_))